In [ ]:
!pip install tweepy pandas nltk textblob

In [ ]:

# Imports
import tweepy
import pandas as pd
import re
import nltk
from textblob import TextBlob
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# you can get this credentials from https://developer.x.com/en/portal/dashboard click on the key icon to generate these

api_key = 'your_api_key'
api_key_secret = 'your_api_key_secret'
access_token = 'your_access_token'
access_token_secret = 'your_acess_token_secret'

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

hf_token = "your_huggingface_token_here"  # get it from here https://huggingface.co/settings/tokens

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForSequenceClassification.from_pretrained(model_name, use_auth_token=hf_token)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import gradio as gr
import pandas as pd
import tweepy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import os

# Setup Twitter API
bearer_token = os.getenv('BEARER_TOKEN')  # Replace with your actual token
client = tweepy.Client(bearer_token=bearer_token)

# Load sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Map BERT labels to readable sentiment
label_map = {
    'LABEL_0': 'negative',
    'LABEL_1': 'neutral',
    'LABEL_2': 'positive'
}

# Helper to fetch tweets
def fetch_tweets_v2(hashtag, max_results=10):
    query = f"{hashtag} -is:retweet lang:en"
    tweets = client.search_recent_tweets(query=query, tweet_fields=["text"], max_results=min(max_results, 10))
    data = [[tweet.text] for tweet in tweets.data] if tweets.data else []
    df = pd.DataFrame(data, columns=['tweet'])
    return df
# Function to analyze tweets (without filter sentiment)
def analyze_hashtag_sentiment(hashtag):
    try:
        df = fetch_tweets_v2(hashtag, max_results=10)
    except Exception as e:
        print(f"[Hashtag Fetch Error] {e}")
        return "No tweets found due to an error."

    if df.empty:
        return "No tweets found."

    sentiments = []
    for text in df['tweet'].tolist():
        try:
            result = sentiment_pipeline(text)[0]
        except Exception as e:
            print(f"[Sentiment Pipeline Error] {e}")
            result = {'label': 'LABEL_1', 'score': 0}
        sentiments.append(result)

    df['sentiment'] = [label_map[s['label']] for s in sentiments]
    df['score'] = [round(s['score'], 3) for s in sentiments]

    overall = df['sentiment'].value_counts().idxmax()
    tweets = '\n\n'.join(df['tweet'].tolist())
    return f"**Overall Sentiment: {overall.upper()}**"


# Function to analyze plain text
def analyze_custom_text_sentiment(text):
    if not text.strip():
        return "⚠️ Please enter some text to analyze."
    try:
        result = sentiment_pipeline(text)[0]
        sentiment = label_map[result['label']]
        score = round(result['score'], 3)
        return f"**Sentiment:** {sentiment.upper()}  \n**Confidence:** {score}"
    except Exception as e:
        print(f"[Custom Text Error] {e}")
        return "❌ Could not analyze sentiment due to an internal error."


# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🐦 Twitter Hashtag Sentiment OR 🔤 Direct Text Analysis")

    with gr.Tab("Analyze Hashtag"):
        hashtag = gr.Textbox(label="Hashtag (e.g., #modi)")
        out1 = gr.Markdown()
        analyze_btn = gr.Button("Analyze Hashtag")
        analyze_btn.click(analyze_hashtag_sentiment, inputs=[hashtag], outputs=out1)

    with gr.Tab("Analyze Custom Text"):
        user_text = gr.Textbox(label="Enter Text")
        out2 = gr.Markdown()
        analyze_text_btn = gr.Button("Analyze Sentiment")
        analyze_text_btn.click(analyze_custom_text_sentiment, inputs=user_text, outputs=out2)

demo.launch(debug=True)


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a5844642f0835f83c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
[Hashtag Fetch Error] 429 Too Many Requests
Too Many Requests
